https://huggingface.co/transformers/quickstart.html

In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# # OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
# import logging
# logging.basicConfig(level=logging.INFO)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [4]:
output.shape

torch.Size([1, 50257])

In [2]:
generated = tokenizer.encode("The Manhattan bridge")
context = torch.tensor([generated])
past = None

for i in range(100):
    #print(i)
    output, past = model(context, past=past)
    token = torch.argmax(output[..., -1, :])

    generated += [token.tolist()]
    context = token.unsqueeze(0)

sequence = tokenizer.decode(generated)

print(sequence)

The Manhattan bridge is a major artery for the city's subway system, and the bridge is one of the busiest in the country.

The bridge is one of the busiest in the country.

The bridge is one of the busiest in the country.

The bridge is one of the busiest in the country.

The bridge is one of the busiest in the country.

The bridge is one of the busiest in the country.

The bridge is one of the busiest in the country


https://github.com/huggingface/transformers/tree/master/examples/language-modeling